Dataset Info

폴더 구조

	•	train_source [폴더]
	•	16개의 PDF 파일
	•	Source에서 여러 질문-답변 쌍을 추출
	•	test_source [폴더]
	•	9개의 PDF 파일
	•	Source에서 여러 질문-답변 쌍을 추출하여, Inference를 위한 질문만 제공

파일 구조

	•	train.csv
	•	SAMPLE_ID : 샘플 별 식별 ID
	•	Source : 질문-답변의 원천이 되는 Source PDF 파일명
	•	Source_path : 질문-답변의 원천이 되는 Source PDF 파일 경로
	•	Question : Source로부터 추출된 질문
	•	Answer : Source로부터 추출된 질문에 대한 정답(답변)
	•	test.csv
	•	SAMPLE_ID : 샘플 별 식별 ID
	•	Source : 질문-답변의 원천이 되는 Source PDF 파일명
	•	Source_path : 질문-답변의 원천이 되는 Source PDF 파일 경로
	•	Question : Source로부터 추출된 질문
	•	sample_submission.csv (제출 양식)
	•	SAMPLE_ID : 샘플 별 식별 ID
	•	Answer : 질문에 대해 생성된 정답(답변)
	•	※ 생성된 정답 중 NULL (빈 값)이 존재하는 경우, 제출이 불가능합니다.

In [1]:
!pip install pymupdf
!pip install langchain
!pip install langchain_community
!pip install bitsandbytes
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 876.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [2]:
import pandas as pd
import os
import unicodedata
from tqdm import tqdm
import pymupdf
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [3]:
def process_pdf(file_path, chunk_size=800, chunk_overlap=50):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    doc = pymupdf.open(file_path)
    text = ''
    for page in doc:
        text += page.get_text()
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_text(text)
    chunks = [Document(page_content=t) for t in chunk_temp]
    return chunks

In [4]:
# def create_vector_db(chunks, model_path="intfloat/multilingual-e5-small"):
#     """FAISS DB 생성"""
#     model_kwargs = {'device': 'cuda'}
#     encode_kwargs = {'normalize_embeddings': True}
#     embeddings = HuggingFaceEmbeddings(
#         model_name=model_path,
#         model_kwargs=model_kwargs,
#         encode_kwargs=encode_kwargs
#     )
#     db = FAISS.from_documents(chunks, embedding=embeddings)
#     return db

def create_vector_db(chunks, model_path="nlpaueb/bert-base-greek-uncased-v1"):
    """
    FAISS DB 생성 함수

    :param chunks: 문서 조각들 (리스트 형식으로 제공되어야 함)
    :param model_path: 사용하고자 하는 벡터 임베딩 모델의 경로
    :return: FAISS 벡터 데이터베이스 객체
    """

    # 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}

    # 임베딩 모델 초기화
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )

    # FAISS 벡터 데이터베이스 생성
    db = FAISS.from_documents(chunks, embedding=embeddings)

    return db

In [5]:
def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)

In [6]:
def process_pdfs_from_dataframe(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()  # 데이터프레임에서 중복되지 않는 PDF 경로 가져오기

    for path in tqdm(unique_paths, desc="Processing PDFs"):
        normalized_path = normalize_path(path)  # 경로 정규화

        # 경로가 상대경로일 경우 base_directory와 결합
        if not os.path.isabs(normalized_path):
            full_path = os.path.join(base_directory, normalized_path.lstrip('./'))
        else:
            full_path = normalized_path

        # 파일이 존재하는지 확인
        if not os.path.isfile(full_path):
            print(f"File not found: {full_path}")
            continue

        pdf_title = os.path.splitext(os.path.basename(full_path))[0]

        # PDF 파일 처리 및 벡터 데이터베이스 생성
        print(f"Processing {pdf_title}...")
        chunks = process_pdf(full_path)
        db = create_vector_db(chunks)
        retriever = db.as_retriever(search_type="mmr", search_kwargs={'k': 3, 'fetch_k': 8})

        # 결과를 딕셔너리에 저장
        pdf_databases[pdf_title] = {
            'db': db,
            'retriever': retriever
        }

    return pdf_databases

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!pip install faiss-gpu
data_path = '/content/drive/MyDrive/데이콘_자료/재정정보_AI검색_알고리즘/'

train_df = pd.read_csv(data_path + 'train.csv')
test_df = pd.read_csv(data_path + 'test.csv')
sample_submission_df = pd.read_csv(data_path + 'sample_submission.csv')

# PDF 데이터베이스 생성
pdf_train_databases = process_pdfs_from_dataframe(train_df, data_path)
pdf_test_databases = process_pdfs_from_dataframe(test_df, data_path)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 24.4 MB/s eta 0:00:00


Processing PDFs:   0%|          | 0/16 [00:00<?, ?it/s]

Processing 1-1 2024 주요 재정통계 1권...


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

...

Processing PDFs:  78%|███████▊  | 7/9 [00:11<00:04,  2.01s/it]

Processing 「FIS 이슈&포커스」 22-2호 《재정성과관리제도》...


Processing PDFs:  89%|████████▉ | 8/9 [00:14<00:02,  2.22s/it]

Processing 「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...


Processing PDFs: 100%|██████████| 9/9 [00:16<00:00,  1.80s/it]


In [9]:
def setup_llm_pipeline():
    # 4비트 양자화 설정
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # 모델 ID
    # model_id = "beomi/llama-2-ko-7b"
    model_id = "rtzr/ko-gemma-2-9b-it"

    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

    # 모델 로드 및 양자화 설정 적용
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True )

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.2,
        return_full_text=False,
        max_new_tokens=128,
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return hf

In [10]:
# !pip install accelerate peft bitsandbytes transformers trl

# LLM 파이프라인
llm = setup_llm_pipeline()

def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

# 결과를 저장할 리스트 초기화
results = []

tokenizer_config.json:   0%|          | 0.00/40.5k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/852 [00:00<?, ?B/s]

...

model-00009-of-00010.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00010-of-00010.safetensors:   0%|          | 0.00/705M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [11]:
# DataFrame의 각 행에 대해 처리
for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Answering Questions"):
    source = normalize_string(row['Source'])
    question = row['Question']

    # 정규화된 키로 데이터베이스 검색
    normalized_keys = {normalize_string(k): v for k, v in pdf_test_databases.items()}
    retriever = normalized_keys[source]['retriever']

    # RAG 체인 구성
    template = """
    다음 정보를 바탕으로 질문에 답하세요:
    {context}

    질문: {question}

    답변:
    """
    prompt = PromptTemplate.from_template(template)

    # RAG 체인 정의
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # 답변 추론
    print(f"Question: {question}")
    full_response = rag_chain.invoke(question)

    print(f"Answer: {full_response}\n")

    # 결과 저장
    results.append({
        "Source": row['Source'],
        "Source_path": row['Source_path'],
        "Question": question,
        "Answer": full_response
    })

Answering Questions:   0%|          | 0/98 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Question: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?


Answering Questions:   1%|          | 1/98 [00:17<27:45, 17.17s/it]

Answer: 2022년 혁신창업사업화자금(융자)의 예산은 2조 300억원입니다. 2022년 혁신창업사업화자금(융자)은 2023년 1월부터 폐지되었습니다. 2023년 1월부터 폐지된 혁신창업사업화자금(융자) 대신 성장공유형 대출방식이 추가되었습니다.



Question: 중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Answering Questions:   2%|▏         | 2/98 [00:33<26:18, 16.44s/it]

...

Answer: 국가결산보고서와 지방자치단체 회계기준은 모두 우발부채를 다루지만, 용어와 회계처리에 있어서 몇 가지 중요한 차이점이 있습니다.

    1. 용어:
        - 국가결산보고서 주석5에서는 '우발사항 및 약정사항'이라는 용어를 사용하며, 이는 우발부채를 포함합니다.
        - 지방자치단체 회계기준은 여전히 '우발손실'이라는 용어를 사용하고

Question: 우발부채란 무엇이며, 그 관리가 왜 중요한가?


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Answering Questions:  99%|█████████▉| 97/98 [25:27<00:18, 18.05s/it]

Answer: 우발부채는 미래에 발생할 가능성이 있는, 하지만 현재 확실하지 않은 의무 또는 부채를 말합니다. 
    예를 들어, 정부가 법적 책임을 져야 할 수 있는 사건이 발생할 가능성이 있는 경우, 이는 우발부채로 
    분류될 수 있습니다. 

    우발부채 관리가 중요한 이유는 다음과 같습니다:

    1. **재정 안정성:** 우발부채는 정부의 재정 건전성에 부정적인 영

Question: 보증이란 무엇이며, 어떤 형태의 보증이 재정상태표에 부채로 기록되는가? 또한 표준화 보증이란 무엇이며, 어떤 목적으로 발행되는가?


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Answering Questions: 100%|██████████| 98/98 [25:45<00:00, 15.77s/it]

Answer: 보증은 재정적 의무를 떠맡는 행위로, 특정 사건이 발생할 경우 발생할 수 있는 손실을 보상하기 위해 제공됩니다. 보증은 재정상태표에 부채로 기록됩니다.

    보증의 종류는 다양하며, 그 중 하나가 표준화 보증(Standard Cost Support, SCS)입니다. SCS는 특정 프로젝트, 특히 공공 프로젝트에서 사용되는 보증 형태입니다. 

    SCS는 프로젝트의 운영 수입이 예상된 수익



In [12]:
# 결과를 CSV 파일로 저장하기 전에 모델에서 빈 값 처리
sample_submission_df['Answer'] = [item['Answer'] for item in results]
sample_submission_df['Answer'] = sample_submission_df['Answer'].fillna("데이콘")  # 빈 값 채우기

# 결과를 CSV 파일로 저장
sample_submission_df.to_csv(os.path.join(data_path, "baseline_submission.csv"), encoding='UTF-8-sig', index=False)

In [17]:
%cd /content/drive/MyDrive/캐글_데이콘_코드/Kaggle-Dacon/
!git config --global user.email 'jik970401@gmail.com'
!git config --global user.name 'jikyoung'

/content/drive/MyDrive/캐글_데이콘_코드/Kaggle-Dacon


In [18]:
!git add Financial_Information_AI_Search_Algorithm.ipynb

In [19]:
!git commit -m '재정정보 AI 검색 알고리즘 경진대회'
!git push

[main c5f944c] 재정정보 AI 검색 알고리즘 경진대회
 1 file changed, 1 insertion(+)
 create mode 100644 Financial_Information_AI_Search_Algorithm.ipynb
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 12 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 42.86 KiB | 3.30 MiB/s, done.
Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
To https://github.com/jikyoung/Kaggle-Dacon.git
   b021fa1..c5f944c  main -> main
